### TGAN
!pip install tgan --user

>>> from tgan.model import TGANModel
>>> tgan = TGANModel(
   ...:     continuous_columns,
   ...:     output='output',
   ...:     max_epoch=5,
   ...:     steps_per_epoch=10000,
   ...:     save_checkpoints=True,
   ...:     restore_session=True,
   ...:     batch_size=200,
   ...:     z_dim=200,
   ...:     noise=0.2,
   ...:     l2norm=0.00001,
   ...:     learning_rate=0.001,
   ...:     num_gen_rnn=100,
   ...:     num_gen_feature=100,
   ...:     num_dis_layers=1,
   ...:     num_dis_hidden=100,
   ...:     optimizer='AdamOptimizer'
   ...: )

In [164]:
import pandas as pd
import numpy as np
import tgan

In [165]:
df = pd.read_excel('datasets/dataset.xlsx', engine='openpyxl')

In [166]:
pd.DataFrame(df)
df.head(7)

,Material,Weight,Temperature,Time,Grinding Time,Initial Weight,Final Weight,Unnamed: 7,Water added
0,veg_waste+bone,50+50,100,20min 0sec,40sec,100,36.0,NaN,40
1,veg_waste+bone,50+50,300,11min 0sec,40sec,120,53.0,NaN,20
2,veg_waste+bone,70+30,300,14min 0sec,20sec,100,25.0,NaN,20
3,veg_waste+bone,60+40,600,5min 0sec,30sec,100,58.0,NaN,0
4,veg_waste+bone,20+80,800,4min 0sec,20sec,98,42.0,NaN,0
5,veg_waste+banana_leaf,80+20,1600,17min 21sec,32sec,100,84.4,NaN,0
6,veg_waste+banana_leaf,40+10,1600,13min 32sec,19sec,50,42.0,NaN,0


In [167]:
df.drop("Unnamed: 7", axis=1, inplace=True)

In [168]:
df[['min', 'sec']] = df.Time.str.rsplit(expand=True)
df.drop('Time', axis=1, inplace=True)

In [169]:
df['Material'] = df.Material.str.replace('peel', 'veg_waste', regex=True)
df[['M1', 'M2']] = df.Material.str.rsplit("+", expand=True)
df['M3'] = df.M2
df[['W1', 'W2']] = df.Weight.str.rsplit("+", expand=True)
df['W3'] = df.W2
df.drop(['Material'], axis=1, inplace=True)

In [177]:
df.drop(['Weight'], axis=1, inplace=True)

In [170]:
mapping = {'veg_waste': 1, 'bone': 2, 'banana_leaf': 3}
df.replace(to_replace="veg_waste",value=1, inplace=True)
df.replace(to_replace="bone",value=2, inplace=True)
df.replace(to_replace="banana_leaf",value=3, inplace=True)
# df.M3.replace(to_replace=2, value=0, inplace=True)

In [171]:
df['Grinding Time'] = df['Grinding Time'].str.rsplit("sec", expand=True)
df['min'] = df['min'].str.rsplit("min", expand=True)
df['sec'] = df['sec'].str.rsplit("sec", expand=True)

In [172]:
df.M2.replace({2:1, 3:0}, inplace=True)

In [175]:
df.M3.replace({2:0, 3:1}, inplace=True)

In [178]:
df

,Temperature,Grinding Time,Initial Weight,Final Weight,Water added,min,sec,M1,M2,M3,W1,W2,W3
0,100,40,100,36.00,40,20,0,1,1,0,50,50,50
1,300,40,120,53.00,20,11,0,1,1,0,50,50,50
2,300,20,100,25.00,20,14,0,1,1,0,70,30,30
3,600,30,100,58.00,0,5,0,1,1,0,60,40,40
4,800,20,98,42.00,0,4,0,1,1,0,20,80,80
5,1600,32,100,84.40,0,17,21,1,0,1,80,20,20
6,1600,19,50,42.00,0,13,32,1,0,1,40,10,10
7,1800,31,100,87.30,0,14,49,1,0,1,80,20,20
8,1800,20,50,39.20,0,10,57,1,0,1,40,10,10
9,2000,34,100,82.50,0,12,06,1,0,1,80,20,20


In [186]:
df.W2.iloc[5:] = 0
df.W3.iloc[:5] = 0

In [187]:
df

,Temperature,Grinding Time,Initial Weight,Final Weight,Water added,min,sec,M1,M2,M3,W1,W2,W3
0,100,40,100,36.00,40,20,0,1,1,0,50,50,0
1,300,40,120,53.00,20,11,0,1,1,0,50,50,0
2,300,20,100,25.00,20,14,0,1,1,0,70,30,0
3,600,30,100,58.00,0,5,0,1,1,0,60,40,0
4,800,20,98,42.00,0,4,0,1,1,0,20,80,0
5,1600,32,100,84.40,0,17,21,1,0,1,80,0,20
6,1600,19,50,42.00,0,13,32,1,0,1,40,0,10
7,1800,31,100,87.30,0,14,49,1,0,1,80,0,20
8,1800,20,50,39.20,0,10,57,1,0,1,40,0,10
9,2000,34,100,82.50,0,12,06,1,0,1,80,0,20


In [188]:
df.to_csv("GAN_data.csv")